In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datatable as dt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense, Flatten, InputLayer, Dropout, Input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, accuracy_score

from warnings import filterwarnings
filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
### set seeds
my_seed = 42

np.random.seed(my_seed)
tf.random.set_seed(my_seed)

## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Introduction</span>

<div style="font-size: 1em; font-family: Verdana">
    <b>Hi,</b><br><br>
    I just wanted to share my baseline-model with you guys.<br>
    I've just recently started getting into 'deep learning' and read a lot of basics.<br>
    This is the reason why I decided to use this month competition to get some practice with Neural-Networks.<br><br>
    Also make sure to check out my EDA for TPS November 2021 <a href="https://www.kaggle.com/mlanhenke/tps-11-simple-basic-eda">here</a>. <br><br>
    <em>If you like this notebook or copy any parts of it please make sure to leave an upvote...</em><br><br>
    <em><b>Thanks for taking some time to stop by and read my notebook!</b></em>
</div>



## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Import Data & Pre-Processing</span>

In [ ]:
### load dataframes
df_train = dt.fread('../input/tabular-playground-series-nov-2021/train.csv').to_pandas()
df_test = dt.fread('../input/tabular-playground-series-nov-2021/test.csv').to_pandas()

sample_submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

### split into X, y
X = df_train.drop(columns=['id','target']).copy()
y = df_train['target'].copy()

X_test = df_test.drop(columns='id').copy()

### standardize data
scaler = StandardScaler()

X = pd.DataFrame(columns=X.columns, data=scaler.fit_transform(X))
X_test = pd.DataFrame(columns=X_test.columns, data=scaler.transform(X_test))

## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Modeling</span>

In [ ]:
### define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=16, 
    verbose=0,
    mode='min', 
    baseline=None, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,
    patience=5,
    mode='min'
)

In [ ]:
### create baseline-model
def get_model():
    inp = Input(shape=X.shape[1], name='input')
    h = Dense(128, activation='swish')(inp)
    h = Dropout(0.25)(h)
    h = Dense(64, activation='swish')(h)
    h = Dropout(0.25)(h)
    h = Dense(32, activation='swish')(h)
    h = Dropout(0.25)(h)
    h = Dense(1, activation='sigmoid')(h)
    
    model = Model(inputs=inp, outputs=h)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-3),
        metrics=['AUC']
    )
    return model

In [ ]:
EPOCHS = 100
BATCH_SIZE = 1024
VERBOSE = 0
N_SPLITS = 10

### cross-validation 
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=my_seed)

scores = {fold:None for fold in range(cv.n_splits)}
predictions = []

for fold, (idx_train, idx_valid) in enumerate(cv.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model = get_model()

    print('**'*20)
    print(f"Fold {fold+1} || Training")
    print('**'*20)
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=VERBOSE,
        callbacks=[
            early_stopping,
            reduce_lr
        ]
    )
    
    scores[fold] = (history.history)
    
    print(f"Fold {fold+1} || Max Validation AUC: {np.max(scores[fold]['val_auc'])}")
    
    prediction = model.predict(X_test, batch_size=BATCH_SIZE).reshape(1,-1)[0]
    predictions.append(prediction)

print('**'*20)
print('Finished Training')
print('**'*20)

overall_auc = [np.max(scores[fold]['val_auc']) for fold in range(cv.n_splits)]
print('Overall Mean AUC: ', np.mean(overall_auc))

## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Evaluation</span>

In [ ]:
### plot train and valid loss over number of epochs
fig, ax = plt.subplots(2, 5, tight_layout=True, figsize=(20,5))
ax = ax.flatten()

for fold in range(cv.n_splits):
    df_eval = pd.DataFrame({'train_loss': scores[fold]['loss'], 'valid_loss': scores[fold]['val_loss']})

    min_train = np.round(np.min(df_eval['train_loss']),5)
    min_valid = np.round(np.min(df_eval['valid_loss']),5)
    delta = np.round(min_valid - min_train,5)
    
    sns.lineplot(
        x=df_eval.index,
        y=df_eval['train_loss'],
        label='train_loss',
        ax = ax[fold]
    )

    sns.lineplot(
        x=df_eval.index,
        y=df_eval['valid_loss'],
        label='valid_loss',
        ax = ax[fold]
    )
    
    ax[fold].set_ylabel('')
    ax[fold].set_xlabel(f"Fold {fold+1}\nmin_train: {min_train}\nmin_valid: {min_valid}\ndelta: {delta}", fontstyle='italic')

sns.despine()

## <span style="background:#818181;padding:0.3em;width:100%;display:block;border-radius:0.1em;color:white;font-family:Monospace">Submission</span>

In [ ]:
def postprocess_separate(submission_df, test_df=None, pure_df=None):
    """Update submission_df so that the predictions for the two sides of the hyperplane don't overlap.
    
    Parameters
    ----------
    submission_df : pandas DataFrame with columns 'id' and 'target'
    test_df : the competition's test data
    pure_df : the competition's original training data
    
    From https://www.kaggle.com/ambrosm/tpsnov21-007-postprocessing
    """
    
    sub = submission_df
    
    if pure_df is None: pure_df = pd.read_csv('../input/november21/train.csv')
    if pure_df.shape != (600000, 102): raise ValueError("pure_df has the wrong shape")
    if test_df is None: test_df = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
    if test_df.shape[0] != submission_df.shape[0] or test_df.shape[1] != 101: raise ValueError("test_df has the wrong shape")

    # Find the separating hyperplane for pure_df, step 1
    # Use an SVM with almost no regularization
    model1 = make_pipeline(StandardScaler(), LinearSVC(C=1e5, tol=1e-7, penalty='l2', dual=False, max_iter=2000, random_state=1))
    model1.fit(pure_df.drop(columns=['id', 'target']), pure_df.target)
    pure_pred = model1.predict(pure_df.drop(columns=['id', 'target']))
    print((pure_pred != pure_df.target).sum(), (pure_pred == pure_df.target).sum()) # 1 599999
    # model1 is not perfect: it predicts the wrong class for 1 of 600000 samples

    # Find the separating hyperplane for pure_df, step 2
    # Fit a second SVM to a subset of the points which contains the support vectors
    pure_pred = model1.decision_function(pure_df.drop(columns=['id', 'target']))
    subset_df = pure_df[(pure_pred > -5) & (pure_pred < 0.9)]
    model2 = make_pipeline(StandardScaler(), LinearSVC(C=1e5, tol=1e-7, penalty='l2', dual=False, max_iter=2000, random_state=1))
    model2.fit(subset_df.drop(columns=['id', 'target']), subset_df.target)
    pure_pred = model2.predict(pure_df.drop(columns=['id', 'target']))
    print((pure_pred != pure_df.target).sum(), (pure_pred == pure_df.target).sum()) # 0 600000
    # model2 is perfect: it predicts the correct class for all 600000 training samples
    
    pure_test_pred = model2.predict(test_df.drop(columns=['id', 'target'], errors='ignore'))
    lmax, rmin = sub[pure_test_pred == 0].target.max(), sub[pure_test_pred == 1].target.min()
    if lmax < rmin:
        print("There is no overlap. No postprocessing needed.")
        return
    # There is overlap. Remove this overlap
    sub.loc[pure_test_pred == 0, 'target'] -= lmax + 1
    sub.loc[pure_test_pred == 1, 'target'] -= rmin - 1
    print(sub[pure_test_pred == 0].target.min(), sub[pure_test_pred == 0].target.max(),
          sub[pure_test_pred == 1].target.min(), sub[pure_test_pred == 1].target.max())

In [ ]:
### average predictions over each fold and create submission file
sample_submission['target'] = np.mean(np.column_stack(predictions), axis=1)
sample_submission.to_csv('./nn_baseline.csv', index=False)

postprocess_separate(sample_submission)
sample_submission.to_csv('./nn_baseline-with-post-processing.csv', index=False)